In [162]:
import os
import json
import pandas as pd
from datetime import datetime
from typing import List, Dict, Tuple

In [163]:
# Load configuration from config.json
with open('../models/config.json', 'r') as f:
    config = json.load(f)

postprocess = config['postprocessing']

# Output and input paths from configuration
OUTPUT_PATH = postprocess['output_path']
INPUT_PATH_S = postprocess['input_path_s']
INPUT_PATH_D = postprocess['input_path_d']

# New Input paths
INPUT_PATH_WS = postprocess['input_path_ws']
INPUT_PATH_HD = postprocess['input_path_hd']
INPUT_PATH_D_OBJ = postprocess['input_path_d_obj']
INPUT_PATH_S_OBJ = postprocess['input_path_s_obj']


# Display the loaded paths
print("Loaded paths:")
print(f"OUTPUT_PATH: {OUTPUT_PATH}")
print(f"INPUT_PATH_S: {INPUT_PATH_S}")
print(f"INPUT_PATH_D: {INPUT_PATH_D}")
print(f"INPUT_PATH_WS: {INPUT_PATH_WS}")
print(f"INPUT_PATH_HD: {INPUT_PATH_HD}")


Loaded paths:
OUTPUT_PATH: ../data/postprocessing/
INPUT_PATH_S: ../data/output/stochastic/timeseries/
INPUT_PATH_D: ../data/output/deterministic/timeseries/
INPUT_PATH_WS: ../data/output/deterministic/cases/scenarios/
INPUT_PATH_HD: ../data/input/demands/


## Filename Generations Functions

In [164]:
def get_ws_solution_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the Wait and See solution objective values CSV file.
    """
    filename = f"d_scenarios_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename

def get_probabilities_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the probabilities JSON file.
    """
    filename = f"reduced_heat_demand_scenarios_{start_date}_to_{end_date}_{period}.json"
    return filename

def get_weighted_deterministic_ts_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the deterministic weighted time series CSV file.
    """
    filename = f"d_weighted_{start_date}_to_{end_date}_{period}_ts.csv"
    return filename


def get_stochastic_objective_values_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the stochastic objective values CSV file.
    """
    filename = f"s_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename



## Data Loading Functions

In [165]:
def load_ws_solution_data(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the Wait and See solution objective values for a specific date.
    """
    filename = get_ws_solution_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded Wait and See solution data from {filepath}")
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()

def load_probabilities(date_str: str, period: str, input_path: str) -> Dict[str, float]:
    """
    Load probabilities from the JSON file for a specific date and period.
    Returns a dictionary {ScenarioName: Probability}
    """
    filename = get_probabilities_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        print(f"Loaded probabilities from {filepath}")
        # Extract probabilities
        probabilities = {}
        for scenario_name, scenario_data in data.items():
            probabilities[scenario_name] = scenario_data.get('Probability', 0)
        return probabilities
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return {}
    
def load_weighted_deterministic_ts(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the deterministic weighted time series data for a specific date.
    """
    filename = get_weighted_deterministic_ts_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded deterministic weighted time series data from {filepath}")
        # Display the first few rows
        display(data.head())
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()

def load_scenario_heat_demands(date_str: str, period: str, input_path: str) -> Dict[str, pd.Series]:
    """
    Load scenario heat demands from the JSON file for a specific date and period.
    Returns a dictionary {ScenarioName: pd.Series of heat demands}
    """
    filename = get_probabilities_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        print(f"Loaded scenario heat demands from {filepath}")
        # For each scenario, create a pandas Series with time steps and heat demands
        scenario_heat_demands = {}
        for scenario_name, scenario_data in data.items():
            # Remove 'Probability' key and adjust time steps to zero-based indexing
            heat_demand_data = {int(k) - 1: v for k, v in scenario_data.items() if k != 'Probability'}
            # Convert to Series and sort by time step
            heat_demand_series = pd.Series(heat_demand_data).sort_index()
            scenario_heat_demands[scenario_name] = heat_demand_series
        return scenario_heat_demands
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return {}
    

def get_weighted_deterministic_objective_value_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the deterministic weighted objective value CSV file.
    """
    filename = f"d_weighted_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename

def load_weighted_deterministic_objective_value(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the deterministic weighted objective value for a specific date.
    """
    filename = get_weighted_deterministic_objective_value_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded deterministic objective value from {filepath}")
        # Display the data
        display(data)
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None
    
def load_stochastic_objective_values(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the stochastic objective values for a specific date.
    """
    filename = get_stochastic_objective_values_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded stochastic objective values from {filepath}")
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()




# Calculations

In [166]:
# List of dates to process
dates_to_process = [
    datetime(2023, 3, 3),
    datetime(2023, 3, 16),
    datetime(2023, 4, 3),
    datetime(2023, 4, 9),
    datetime(2023, 5, 14),
    datetime(2023, 5, 15),
    datetime(2023, 6, 19),
    datetime(2023, 6, 22),
    datetime(2023, 6, 26),
    # Add mo
]

# Period for the analysis
period = 'day'


## Compute weighted Wait-and-See Solution

In [167]:
# Initialize a list to store the results
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")
    
    # Load Wait and See solution data
    ws_data = load_ws_solution_data(date_str, period, INPUT_PATH_WS)
    if ws_data.empty:
        print(f"No Wait and See solution data available for date {date_str}.")
        continue
    
    # Display the loaded Wait and See solution data
    print("Wait and See Objective Values:")
    display(ws_data)
    
    # Load probabilities
    probabilities = load_probabilities(date_str, period, INPUT_PATH_HD)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        continue
    
    # Display the probabilities
    print("Probabilities:")
    probabilities_df = pd.DataFrame.from_dict(probabilities, orient='index', columns=['Probability'])
    display(probabilities_df)
    
    # Merge ws_data with probabilities
    ws_data['Probability'] = ws_data['Scenario'].map(probabilities)
    
    # Check for missing probabilities
    if ws_data['Probability'].isnull().any():
        print("Some scenarios do not have associated probabilities.")
        display(ws_data[ws_data['Probability'].isnull()])
        continue
    
    # Compute weighted objective value
    ws_data['WeightedObjective'] = ws_data['ObjectiveValue'] * ws_data['Probability']
    
    # Print intermediate calculations
    print("Intermediate Calculations:")
    display(ws_data)
    
    # Sum the weighted objectives
    weighted_objective_value = ws_data['WeightedObjective'].sum()
    
    # Store the result
    results.append({
        'Date': date.strftime('%Y-%m-%d'),
        'Weighted Objective Value': weighted_objective_value
    })



Processing date: 20230303
Loaded Wait and See solution data from ../data/output/deterministic/cases/scenarios/d_scenarios_20230303_to_20230303_day_obj.csv
Wait and See Objective Values:


,Scenario,ObjectiveValue
0,Scenario1,-285.246990
1,Scenario2,-278.041077
2,Scenario3,-283.568071
3,Scenario4,-280.996862
4,Scenario5,-281.218684
5,Scenario6,-281.296879
6,Scenario7,-280.592415
7,Scenario8,-277.409991
8,Scenario9,-285.322021
9,Scenario10,-280.486241


Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230303_to_20230303_day.json
Probabilities:


,Probability
Scenario1,0.213933
Scenario2,0.085874
Scenario3,0.060216
Scenario4,0.115567
Scenario5,0.098109
Scenario6,0.064584
Scenario7,0.078224
Scenario8,0.061210
Scenario9,0.061475
Scenario10,0.160809


Intermediate Calculations:


...

,Scenario,ObjectiveValue
0,Scenario1,-64.831230
1,Scenario2,-92.425116
2,Scenario3,-141.336986
3,Scenario4,-137.091733
4,Scenario5,-120.619765
5,Scenario6,-96.084127
6,Scenario7,-113.890938
7,Scenario8,-108.261855
8,Scenario9,-118.975175
9,Scenario10,-102.084627


Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230626_to_20230626_day.json
Probabilities:


,Probability
Scenario1,0.008036
Scenario2,0.001415
Scenario3,0.000252
Scenario4,0.031910
Scenario5,0.359087
Scenario6,0.507279
Scenario7,0.001421
Scenario8,0.000250
Scenario9,0.000016
Scenario10,0.090334


Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-64.831230,0.008036,-0.520995
1,Scenario2,-92.425116,0.001415,-0.130789
2,Scenario3,-141.336986,0.000252,-0.035616
3,Scenario4,-137.091733,0.031910,-4.374607
4,Scenario5,-120.619765,0.359087,-43.313014
5,Scenario6,-96.084127,0.507279,-48.741453
6,Scenario7,-113.890938,0.001421,-0.161827
7,Scenario8,-108.261855,0.000250,-0.027087
8,Scenario9,-118.975175,0.000016,-0.001873
9,Scenario10,-102.084627,0.090334,-9.221683


#### Final Results Wait-and-See Solution

In [168]:
# Create a DataFrame of results
results_df = pd.DataFrame(results)
results_df['Weighted Objective Value'] = results_df['Weighted Objective Value'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df)



Final Results:


,Date,Weighted Objective Value
0,2023-03-03,-281.780836
1,2023-03-16,-311.669426
2,2023-04-03,-331.505967
3,2023-04-09,-347.875782
4,2023-05-14,-157.262672
5,2023-05-15,-152.972768
6,2023-06-19,-109.554857
7,2023-06-22,-108.455781
8,2023-06-26,-106.528944


## Compute Heat Demand Difference for the calculation of EEV

In [169]:
def compute_costs_from_differences(date_str: str, period: str, input_path_det_ts: str, input_path_hd: str):
    """
    Compute the differences and costs between deterministic and scenario heat demands for a specific date.
    Returns:
        - total_costs_df: DataFrame containing total costs per scenario.
        - weighted_total_cost: The total weighted cost for the date.
    """
    # Load deterministic weighted time series data
    det_ts_data = load_weighted_deterministic_ts(date_str, period, input_path_det_ts)
    if det_ts_data.empty:
        print(f"No deterministic weighted time series data available for date {date_str}.")
        return None, None

    # Extract the deterministic heat demand
    det_heat_demand = det_ts_data['heat_demand']
    det_heat_demand.name = 'deterministic_heat_demand'

    # Load scenario heat demands
    scenario_heat_demands = load_scenario_heat_demands(date_str, period, input_path_hd)
    if not scenario_heat_demands:
        print(f"No scenario heat demands available for date {date_str}.")
        return None, None

    # Load probabilities
    probabilities = load_probabilities(date_str, period, input_path_hd)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        return None, None

    # Create a DataFrame to store costs
    costs_df = pd.DataFrame({
        't': det_ts_data['t'],
        'deterministic_heat_demand': det_heat_demand
    })

    total_costs = {}
    weighted_total_cost = 0.0

    # For each scenario, compute differences, costs, and total costs
    for scenario_name, scenario_hd_series in scenario_heat_demands.items():
        # Align scenario_hd_series with det_heat_demand
        scenario_hd_series = scenario_hd_series.reindex(det_heat_demand.index)
        scenario_hd_series.name = f"{scenario_name}_heat_demand"

        # Compute difference
        difference = det_heat_demand - scenario_hd_series
        difference.name = f"{scenario_name}_difference"

        # Compute cost per time step
        cost = difference.abs() * 0.01
        cost.name = f"{scenario_name}_cost"

        # Sum costs over time steps
        total_cost = cost.sum()

        # Get probability for the scenario
        probability = probabilities.get(scenario_name, 0)

        # Compute weighted cost
        weighted_cost = total_cost * probability

        # Add to total weighted cost
        weighted_total_cost += weighted_cost

        # Store total cost per scenario
        total_costs[scenario_name] = total_cost

        # Optionally, add to costs_df for detailed output
        costs_df[f"{scenario_name}_cost"] = cost.values

    # Create DataFrame for total costs per scenario
    total_costs_df = pd.DataFrame(list(total_costs.items()), columns=['Scenario', 'Total Cost'])

    # Display total costs per scenario
    print(f"Total Costs for date {date_str}:")
    display(total_costs_df)

    print(f"Weighted Total Cost for date {date_str}: {weighted_total_cost:.6f}")

    return total_costs_df, weighted_total_cost


In [170]:
# Input paths
input_path_det_ts = INPUT_PATH_D  # Path to deterministic weighted time series
input_path_hd = INPUT_PATH_HD     # Path to heat demand scenarios

# Dictionary to store total costs for each date
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")

    # Compute costs and weighted total cost
    total_costs_df, weighted_total_cost = compute_costs_from_differences(
        date_str=date_str,
        period=period,
        input_path_det_ts=input_path_det_ts,
        input_path_hd=input_path_hd
    )

    if weighted_total_cost is not None:
        # Load deterministic objective value
        det_obj_value = load_weighted_deterministic_objective_value(date_str, period, INPUT_PATH_D_OBJ)
        if det_obj_value is not None:
            # det_obj_value is a DataFrame with one value
            det_obj_value = det_obj_value['ObjectiveValue'].iloc[0]
            print(f"Deterministic Objective Value for date {date_str}: {det_obj_value}")

            # Compute total cost
            total_cost = det_obj_value + weighted_total_cost

            # Store the result
            results.append({
                'Date': date.strftime('%Y-%m-%d'),
                'Total Cost': total_cost
            })

            print(f"Total Cost for date {date_str}: {total_cost:.6f}")
        else:
            print(f"Could not load deterministic objective value for date {date_str}.")
    else:
        print(f"Could not compute weighted total cost for date {date_str}.")


Processing date: 20230303
Loaded deterministic weighted time series data from ../data/output/deterministic/timeseries/d_weighted_20230303_to_20230303_day_ts.csv


,t,GAS_PRICE,POWER_PRICE,HEAT_PRICE,heat_demand,heat_storage1.initial_soc,chp1.bin,chp1.power,chp1.gas,chp1.heat,...,heat_grid.heat_supply,heat_grid.heat_feedin,arc01_expanded.power,arc02_expanded.power,arc03_expanded.heat,arc04_expanded.heat,arc05_expanded.heat,arc06_expanded.gas,arc07_expanded.gas,arc08_expanded.heat
0,1,0.1543,0.251,0.105,141.657945,320.0,1.0,37.218589,117.039587,68.0,...,0.0,141.657945,37.218589,37.218589,68.0,68.0,0.000000,0.000000,117.039587,5.657945
1,2,0.1543,0.251,0.105,146.058094,320.0,1.0,37.218589,117.039587,68.0,...,0.0,146.058094,37.218589,37.218589,68.0,68.0,0.000000,0.000000,117.039587,10.058094
2,3,0.1543,0.251,0.105,234.265871,320.0,1.0,37.218589,117.039587,68.0,...,0.0,234.265871,37.218589,37.218589,68.0,68.0,85.200000,78.308824,117.039587,13.065871
3,4,0.1543,0.251,0.105,377.818933,320.0,1.0,37.218589,117.039587,68.0,...,0.0,377.818933,37.218589,37.218589,68.0,68.0,141.400846,131.570620,117.039587,100.418087
4,5,0.1543,0.251,0.105,277.248569,320.0,1.0,37.218589,117.039587,68.0,...,0.0,277.248569,37.218589,37.218589,68.0,68.0,141.248569,131.426306,117.039587,0.000000


Loaded scenario heat demands from ../data/input/demands/reduced_heat_demand_scenarios_20230303_to_20230303_day.json
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230303_to_20230303_day.json
Total Costs for date 20230303:


,Scenario,Total Cost
0,Scenario1,3.442595
1,Scenario2,3.784239
2,Scenario3,3.392275
3,Scenario4,4.061540
4,Scenario5,4.209550
5,Scenario6,3.544443
6,Scenario7,3.798941
7,Scenario8,4.023848
8,Scenario9,3.618931
9,Scenario10,2.943745


Weighted Total Cost for date 20230303: 3.616329
Loaded deterministic objective value from ../data/output/deterministic/objectives/d_weighted_20230303_to_20230303_day_obj.csv


...

Loaded scenario heat demands from ../data/input/demands/reduced_heat_demand_scenarios_20230626_to_20230626_day.json
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230626_to_20230626_day.json
Total Costs for date 20230626:


,Scenario,Total Cost
0,Scenario1,6.491448
1,Scenario2,6.373410
2,Scenario3,8.635586
3,Scenario4,5.414318
4,Scenario5,3.314589
5,Scenario6,2.254107
6,Scenario7,6.259954
7,Scenario8,7.313466
8,Scenario9,7.652818
9,Scenario10,6.793851


Weighted Total Cost for date 20230626: 3.194379
Loaded deterministic objective value from ../data/output/deterministic/objectives/d_weighted_20230626_to_20230626_day_obj.csv


,ObjectiveValue
0,-101.891601


Deterministic Objective Value for date 20230626: -101.8916012290719
Total Cost for date 20230626: -98.697222


## EEV 

In [171]:

# Create a DataFrame of results
results_df_eev = pd.DataFrame(results)
results_df_eev['Total Cost'] = results_df_eev['Total Cost'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df_eev)



Final Results:


,Date,Total Cost
0,2023-03-03,-278.448919
1,2023-03-16,-308.692221
2,2023-04-03,-327.843262
3,2023-04-09,-344.289766
4,2023-05-14,-151.028305
5,2023-05-15,-144.744360
6,2023-06-19,-116.584602
7,2023-06-22,-119.956988
8,2023-06-26,-98.697222


In [172]:
def compute_weighted_stochastic_objective_value(date_str: str, period: str, input_path_s: str, input_path_hd: str) -> float:
    """
    Compute the weighted stochastic objective value for a specific date.
    """
    # Load stochastic objective values
    sto_obj_values_df = load_stochastic_objective_values(date_str, period, input_path_s)
    if sto_obj_values_df.empty:
        print(f"No stochastic objective values data available for date {date_str}.")
        return None

    # Load probabilities
    probabilities = load_probabilities(date_str, period, input_path_hd)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        return None

    # Clean up column names if necessary
    # Sometimes the first column might have a stray ':' character
    sto_obj_values_df.columns = [col.strip(':') for col in sto_obj_values_df.columns]

    # Merge sto_obj_values_df with probabilities
    sto_obj_values_df['Probability'] = sto_obj_values_df['Scenario'].map(probabilities)

    # Check for missing probabilities
    if sto_obj_values_df['Probability'].isnull().any():
        print("Some scenarios do not have associated probabilities.")
        display(sto_obj_values_df[sto_obj_values_df['Probability'].isnull()])
        return None

    # Compute weighted objective value
    sto_obj_values_df['WeightedObjective'] = sto_obj_values_df['ObjectiveValue'] * sto_obj_values_df['Probability']

    # Display intermediate calculations
    print("Intermediate Calculations:")
    display(sto_obj_values_df)

    # Sum the weighted objectives
    weighted_objective_value = sto_obj_values_df['WeightedObjective'].sum()

    return weighted_objective_value


In [173]:
# Input paths
input_path_s_obj = INPUT_PATH_S_OBJ  # Path to stochastic objective values
input_path_hd = INPUT_PATH_HD  # Path to heat demand scenarios (probabilities)

# Initialize a list to store the results
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")

    # Compute weighted stochastic objective value
    weighted_sto_obj_value = compute_weighted_stochastic_objective_value(
        date_str=date_str,
        period=period,
        input_path_s=input_path_s_obj,
        input_path_hd=input_path_hd
    )

    if weighted_sto_obj_value is not None:
        # Store the result
        results.append({
            'Date': date.strftime('%Y-%m-%d'),
            'Weighted Stochastic Objective Value': weighted_sto_obj_value
        })
        print(f"Weighted Stochastic Objective Value for date {date_str}: {weighted_sto_obj_value:.6f}")
    else:
        print(f"Could not compute weighted stochastic objective value for date {date_str}.")


Processing date: 20230303
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230303_to_20230303_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230303_to_20230303_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-268.198006,0.213933,-57.376294
1,Scenario2,-268.123682,0.085874,-23.024748
2,Scenario3,-267.439334,0.060216,-16.104156
3,Scenario4,-267.278828,0.115567,-30.888494
4,Scenario5,-267.576815,0.098109,-26.251561
5,Scenario6,-268.387175,0.064584,-17.333617
6,Scenario7,-266.357883,0.078224,-20.835648
7,Scenario8,-267.630136,0.061210,-16.381549
8,Scenario9,-267.198345,0.061475,-16.426117
9,Scenario10,-267.502840,0.160809,-43.016856


Weighted Stochastic Objective Value for date 20230303: -267.639041

Processing date: 20230316
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230316_to_20230316_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230316_to_20230316_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-298.678107,0.093211,-27.839944
1,Scenario2,-298.803038,0.041917,-12.524815
2,Scenario3,-297.714182,0.073168,-21.783027
3,Scenario4,-299.036387,0.164225,-49.109336
4,Scenario5,-298.610576,0.096919,-28.941104
5,Scenario6,-298.743012,0.165247,-49.366534
6,Scenario7,-298.454066,0.048252,-14.401132
7,Scenario8,-298.647855,0.114138,-34.087146
8,Scenario9,-298.677781,0.088206,-26.345239
9,Scenario10,-298.610365,0.114716,-34.255495


Weighted Stochastic Objective Value for date 20230316: -298.653771

Processing date: 20230403
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230403_to_20230403_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230403_to_20230403_day.json
Intermediate Calculations:


...

Weighted Stochastic Objective Value for date 20230619: -127.764703

Processing date: 20230622
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230622_to_20230622_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230622_to_20230622_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-130.796812,0.135874,-17.771877
1,Scenario2,-132.495667,0.082486,-10.929019
2,Scenario3,-130.818436,0.127544,-16.685105
3,Scenario4,-132.828819,0.194525,-25.838566
4,Scenario5,-130.713407,0.081249,-10.620394
5,Scenario6,-132.284410,0.108861,-14.400629
6,Scenario7,-131.622467,0.059450,-7.825010
7,Scenario8,-131.987819,0.066503,-8.777565
8,Scenario9,-132.017060,0.081728,-10.789444
9,Scenario10,-131.361935,0.061779,-8.115467


Weighted Stochastic Objective Value for date 20230622: -131.753075

Processing date: 20230626
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230626_to_20230626_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230626_to_20230626_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-120.956495,0.008036,-0.972028
1,Scenario2,-122.179775,0.001415,-0.172894
2,Scenario3,-121.330432,0.000252,-0.030574
3,Scenario4,-122.315400,0.031910,-3.903094
4,Scenario5,-123.575223,0.359087,-44.374281
5,Scenario6,-122.924965,0.507279,-62.357245
6,Scenario7,-121.638032,0.001421,-0.172835
7,Scenario8,-121.407041,0.000250,-0.030376
8,Scenario9,-120.863951,0.000016,-0.001902
9,Scenario10,-122.097052,0.090334,-11.029479


Weighted Stochastic Objective Value for date 20230626: -123.044709


In [174]:
# Create a DataFrame of results
results_df_rp = pd.DataFrame(results)
results_df_rp['Weighted Stochastic Objective Value'] = results_df_rp['Weighted Stochastic Objective Value'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df_rp)



Final Results:


,Date,Weighted Stochastic Objective Value
0,2023-03-03,-267.639041
1,2023-03-16,-298.653771
2,2023-04-03,-317.622410
3,2023-04-09,-336.003228
4,2023-05-14,-173.570396
5,2023-05-15,-168.338485
6,2023-06-19,-127.764703
7,2023-06-22,-131.753075
8,2023-06-26,-123.044709


In [175]:
# Save the EEV results to a CSV file
filename = "eev.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df_eev.to_csv(output_filepath, index=False)


# Save the WS results to a CSV file
filename = "ws.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df.to_csv(output_filepath, index=False)


# Save the RP results to a CSV file
filename = "rp.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df_rp.to_csv(output_filepath, index=False)
